In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("EBOOK-USER-LIKE.csv", delimiter=";")
df.head()

,id,user_id,judul,tahun_terbit,deskripsi,created_at
0,2381,221,WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDA...,1981,WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDA...,21/02/22 14.04
1,3407,526,Monumen Pancasila Sakti,1984,Monumen Pancasila Sakti Penjelasan tentang Mon...,21/02/22 14.05
2,1436,522,113 OPTIMALISASI KEPEMIMPINAN DANDIM DALAM RAN...,2020,Taskap yang dibuat dengan Judul 113 OPTIMALISA...,23/02/22 11.46
3,1436,276,113 OPTIMALISASI KEPEMIMPINAN DANDIM DALAM RAN...,2020,Taskap yang dibuat dengan Judul 113 OPTIMALISA...,23/02/22 11.46
4,1427,522,104 OPTIMALISASI KEMAMPUAN SATGASPAMTAS RI-PNG...,2020,Taskap yang dibuat dengan Judul 104 OPTIMALISA...,23/02/22 11.55


In [17]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251 entries, 0 to 1250
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1251 non-null   int64 
 1   user_id       1251 non-null   int64 
 2   judul         1251 non-null   object
 3   tahun_terbit  1251 non-null   int64 
 4   deskripsi     1235 non-null   object
 5   created_at    1213 non-null   object
dtypes: int64(3), object(3)
memory usage: 58.8+ KB


In [18]:
def print_description(index):
    example = df[df.index == index][['deskripsi', 'judul', 'user_id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1]) 
        print('User ID:', example[2]) 

In [19]:
print_description(0)

WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDAYAAN) BUKU III
Nama: WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDAYAAN) BUKU III
User ID: 221


In [21]:
import nltk
nltk.download('stopwords')

#Text Preprocessing
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['desc_clean'] = df['deskripsi'].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbookpro2020m1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
df.head()

,id,user_id,judul,tahun_terbit,deskripsi,created_at,desc_clean
0,2381,221,WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDA...,1981,WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDA...,21/02/22 14.04,wawasan nusantara dalam pendidikan dan kebuday...
1,3407,526,Monumen Pancasila Sakti,1984,Monumen Pancasila Sakti Penjelasan tentang Mon...,21/02/22 14.05,monumen pancasila sakti penjelasan tentang mon...
2,1436,522,113 OPTIMALISASI KEPEMIMPINAN DANDIM DALAM RAN...,2020,Taskap yang dibuat dengan Judul 113 OPTIMALISA...,23/02/22 11.46,taskap yang dibuat dengan judul 113 optimalisa...
3,1436,276,113 OPTIMALISASI KEPEMIMPINAN DANDIM DALAM RAN...,2020,Taskap yang dibuat dengan Judul 113 OPTIMALISA...,23/02/22 11.46,taskap yang dibuat dengan judul 113 optimalisa...
4,1427,522,104 OPTIMALISASI KEMAMPUAN SATGASPAMTAS RI-PNG...,2020,Taskap yang dibuat dengan Judul 104 OPTIMALISA...,23/02/22 11.55,taskap yang dibuat dengan judul 104 optimalisa...


In [23]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['desc_clean', 'judul', 'user_id']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Judul:', example[1])
        print('User ID:', example[2])

In [24]:
print_description_clean(0)

wawasan nusantara dalam pendidikan dan kebudayaan buku iii
Judul: WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDAYAAN) BUKU III
User ID: 221


In [25]:
#TF-IDF & Cosine Similarity
df.set_index('judul', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.00202217, 0.00360214, ..., 0.        , 0.        ,
        0.        ],
       [0.00202217, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00360214, 0.        , 1.        , ..., 0.        , 0.08532565,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.08532565, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [26]:
# Set index utama di kolom 'judul'
indices = pd.Series(df.index)
indices[:50]

0     WAWASAN NUSANTARA (DALAM PENDIDIKAN DAN KEBUDA...
1                               Monumen Pancasila Sakti
2     113 OPTIMALISASI KEPEMIMPINAN DANDIM DALAM RAN...
3     113 OPTIMALISASI KEPEMIMPINAN DANDIM DALAM RAN...
4     104 OPTIMALISASI KEMAMPUAN SATGASPAMTAS RI-PNG...
5     104 OPTIMALISASI KEMAMPUAN SATGASPAMTAS RI-PNG...
6                                 PASUKAN ELIT DI DUNIA
7                Hafal Al Quran Dan Lancar Seumur Hidup
8              Obrolan Nusantara: Menyambut Indonesia""
9                                     10 Kisah Genocide
10    Organisasi dan Tugas Direktorat Kesehatan Angk...
11                                 Manajemen Logistik 2
12    Organisasi dan Tugas Peralatan kodam (Orgas Pa...
13    Organisasi dan Tugas Direktorat Peralatan Angk...
14    404 PERAN SATUAN PERALATAN DALAM PEMELIHARAAN ...
15                        BUKU PETUNJUK INDUK PERALATAN
16                                   Ekonomi Kerakyatan
17    Organisasi dan Tugas Perhubungan Komando D

In [50]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_book = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_book.append(list(df.index)[i])
        
    return recommended_book

In [54]:
recommendations('Darurat Terorisme')

['Darurat Terorisme',
 '131 OPTIMALISASI KEMAMPUAN DENINTELDAM IV DIPONEGORO DALAM RANGKA MENCEGAH AKSI TERORISME',
 '60 OPTIMALISASI PERAN KODIM DALAM MENGATASI AKSI TERORISME DI INDONESIA',
 'Hukum Internasional Suatu Pengantar',
 'Hukum Internasional Suatu Pengantar',
 'Politik Identitas',
 '86 OPTIMALISASI PERAN PERWIRA HUKUM TNI AD DALAM PENYELNGGARAAN DUKUNG HUKUM PADA TUGAS OPERASI MILITER',
 'TERORISME SEBAGAI KEJAHATAN TERHADAP KEMANUSIAAN',
 'Terorisme ideologi penebar ketakutan',
 '324 OPTIMALISASI PERAN KOMANDAN SATUAN GUAN MENCEGAH TERJADINYA TINDAK PELANGGARAN HUKUM']